## Advent of code 2018 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers

import zio

In [ ]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

In [ ]:
# 2018 day 4
# start_ts=1702286032
# mv ~/Downloads/input* data_src/2018-day-4-input.txt
# big input file looks like: time log
# idea: part 1 parse ..., then per guard count minutes asleep and most-asleep-minute

sample2='''
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
'''

sample1=open('data_src/2018-day-4-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
lines=sorted(lines)

def parse_line(line, guard):
    '''parse [1518-11-01 23:58] Guard #99 begins shift into minute,guard,status'''
    tup=line.split(']')
    assert (result:= re.match(r'(\d+)-(\d+)-(\d+)\s(\d+):(\d+)', tup[0][1:]))
    mdat=result.group(1,2,3,4,5)
    mdat=[ int(n) for n in mdat ]
    if 'asleep' in tup[1]:
        status='asleep'
        assert guard is not None
    elif 'wakes' in tup[1]:
        status='wakes'
        assert guard is not None
    else:
        assert (result:= re.match(r'\s*Guard #(\d+) begins shift', tup[1]))
        guard=int(result.group(1))
        status='begins'
    assert mdat[0]==1518
    minute=31*mdat[1]+mdat[2]
    minute=24*60*minute+24*mdat[3]+mdat[4]
    #print(f'{mdat=}, {minute=}, {status=}, {guard=}')
    return minute, guard, status
    
# part 1
guard=None
asleep=False
lastmin=None
sleepcount=collections.Counter() # maps guard id to total asleep minutes
sleepmin={} # maps guard id to map of hour-minute to asleep count
for line in lines:
    oldguard=guard
    minute,guard,status=parse_line(line, guard)
    if asleep and lastmin: # old guard was asleep
        sleepcount[oldguard]+=minute-lastmin
        slmap=sleepmin.setdefault(oldguard, collections.Counter())
        for min in range(lastmin, minute):
            slmap[min % 60]+=1
    if status=='begins' or status=='wakes':
        asleep=False
    else:
        assert status=='asleep'
        asleep=True
    lastmin=minute
# now report
#print(f'{sleepcount=}')
#for k,v in sleepmin.items():
#    print(f'{guard=}, sleepmin={v}')
princess_sleep=max(sleepcount.values())
princesses=[ k for k,v in sleepcount.items() if v==princess_sleep ]
assert len(princesses)==1
princess=princesses[0]
slmap=sleepmin[princess]
maxminsleep=max(slmap.values())
maxmins=[ k for k,v in slmap.items() if v==maxminsleep ]
assert len(maxmins)==1
score=princess*maxmins[0]
print(f'part 1: {score=}')

# part 2
minprin=None # minute princess
mpcount=None # slept how many on fav minute
mpmin=None # fav minute to sleep on
for guard,slmap in sleepmin.items():
    maxminsleep=max(slmap.values())
    maxmins=[ k for k,v in slmap.items() if v==maxminsleep ]
    if mpcount is None or maxminsleep>mpcount:
        minprin=guard
        mpcount=maxminsleep
        mpmin=maxmins[0]
score=minprin*mpmin
print(f'part 2: {score=}')

In [ ]:
# 2018 day 3
# start_ts=1701448256
# mv ~/Downloads/input* data_src/2018-day-3-input.txt
# big input file looks like: list of coords and sizes
# idea: part 1 parse with regex, then paint into dict board, counting claims
#  part 2: to part 1 add a last claim id per cell and a set of intact claims that are discarded 
#  when overwriting an existing claim

sample2='''
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2
'''

sample1=open('data_src/2018-day-3-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ result.group(1, 2, 3, 4, 5) for s in lines if \
      (result:= re.match(r'\#(\d+)[\s\@]+([\d\-]+),([\d\-]+):\s*([\d\-]+)x([\d\-]+)', s)) ]
data=[ [ int(n) for n in tup ] for tup in data ]

# part 1 - count overlapping squares
board=collections.Counter() # maps cell x,y to claim count
claimid={} # maps cell x,y to claim id
intactclaims=set() # set of claim id
for tup in data:
    indx,x0,y0,w,h=tup
    intactclaims.add(indx)
    for x in range(x0, x0+w):
        for y in range(y0, y0+h):
            if (x,y) in claimid: # already claimed, we're spoiling it and ourselves
                oldcid=claimid[(x,y)]
                intactclaims.discard(oldcid)
                intactclaims.discard(indx)
            board[(x,y)]+=1
            claimid[(x,y)]=indx
overlap=0
for k,v in board.items():
    if v>1:
        overlap+=1
print(f'part 1: {overlap=}')

# part 2 - intact claims
print(f'part 2: {intactclaims=}')

In [ ]:
# 2018 day 2
# start_ts=1701377203
# mv ~/Downloads/input* data_src/2018-day-2-input.txt
# big input file looks like: list of strings
# idea: part 1 parse as list, then per line / word count letter frequencies, then count words
#  which have a letter with a certain frequency, then multiply those two values
# part 2: totally unrelated to part 1 (?) generate each pair of words, count different letters in
#  the same position, collect common letters, if only one different that's the answer

def count_freqs_n(strs, n):
    '''return number of strings with a letter that occurs exactly n times'''
    cnt=0
    for s in strs:
        counter=collections.Counter()
        for c in s:
            counter[c]+=1
        for v in counter.values():
            if v==n:
                cnt+=1
                break
    return cnt

sample1=open('data_src/2018-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
a=count_freqs_n(lines, 2)
b=count_freqs_n(lines, 3)
n=a*b
print(f'part 1: {n=}')

# part 2
for i, sa in enumerate(lines):
    for sb in lines[:i]:
        assert len(sa)==len(sb)
        common=''
        diffcnt=0
        for j in range(len(sa)):
            if sa[j]==sb[j]:
                common=common+sa[j]
            else:
                diffcnt+=1
        if diffcnt==1:
            print(f'part 2: {common=}')


In [ ]:
# 2018 day 1
# start_ts=1701341821
# mv ~/Downloads/input* data_src/2018-day-1-input.txt
# big input file looks like: pos. and negative numbers
# idea: part 1 parse then add, skipping leading + (wasn't actually needed)
# part 2: keep looping, keeping reached frequencies in a set

sample1=open('data_src/2018-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# part 1
n=0
for s in lines:
    if s[0]=='+':
        n+=int(s[1:])
    else:
        n+=int(s)
print(f'part 1: {n=}')

# part 2
n=0
seenfreq=set()
stopped=False
while not stopped:
    for s in lines:
        if s[0]=='+':
            n+=int(s[1:])
        else:
            n+=int(s)
        if n in seenfreq:
            stopped=True
            break
        seenfreq.add(n)
print(f'part 2: {n=}')

In [ ]:
# TEMPLATE
# 2018 day 1
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2018-day-1-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2018-day-1-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed